<h1 align="center">Concurrency Problems</h1>

In [12]:
from threading import Thread, Semaphore
import time

class myThread(Thread):
    def __init__(self, func, *args):
        Thread.__init__(self)
        self.func = func
        self.args = args
        self.start()
    
    def run(self):
        self.func(*self.args)

### Producer Consumer Problem

In [15]:
class ProducerConsumer():
    def __init__(self):
        self.buff = 0
        self.n = 10
        self.size = 5
        self.mutex = Semaphore(1)
        self.full = Semaphore(0)
        self.empty = Semaphore(self.size)

    def produce(self):
        for _ in range(self.n):
            self.empty.acquire()
            self.mutex.acquire()
            self.buff+=1
            print("Item Produced || BUffer = ", self.buff)
            self.mutex.release()
            self.full.release()
            time.sleep(0.2)

    def consume(self):
        for _ in range(self.n):
            self.full.acquire()
            self.mutex.acquire()
            self.buff -= 1
            print("Item Consumed || BUffer = ", self.buff)
            self.mutex.release()
            self.empty.release()
            time.sleep(0.2)

pc = ProducerConsumer()
t2 = myThread(pc.produce)
t1 = myThread(pc.consume)
t1.start()
t2.start()

Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0
Item Produced || BUffer =  1
Item Consumed || BUffer =  0


### Reader Writer Problem

In [16]:
class ReaderWriter:
    def __init__(self):
        
        # if someone have the system .. you cannot write for sure
        # that means if there are any readers or if there is even a single writer
        self.n = 4
        self.readers = Semaphore(0)
        self.canWrite = Semaphore(1)
        self.mutex = Semaphore(1) # to be applied on readers variable
    
    def write(self):
        for i in range(self.n):
            self.canWrite.acquire()
            print("Writing...  ")
            time.sleep(0.5)
            self.canWrite.release()
            time.sleep(0.5)
    
    def read(self):
        for i in range(self.n):
            if self.readers._value == 0:
                self.canWrite.acquire()
            self.mutex.acquire()
            self.readers.release()
            self.mutex.release()
            print("Reading .... ")
            time.sleep(0.5)
            self.mutex.acquire()
            self.readers.acquire()
            self.mutex.release()
            if self.readers._value == 0:
                self.canWrite.release()
                
rw = ReaderWriter()
myThread(rw.write).start()
myThread(rw.read).start()
        

Writing...  
Reading .... 
Reading .... 
Reading .... 
Reading .... 
Writing...  
Writing...  
Writing...  


### Dining Philosophers

In [22]:
class DiningPhilosopher:
    def __init__(self, n):
        self.forks = [Semaphore(1) for i in range(5)]
        self.lock = Semaphore(4)
        self.n = n
    def eat(self, phs):
        for i in range(self.n):
            with self.lock:
                with self.forks[phs], self.forks[(phs+1)%5]:
                    print(f"Philosopher {phs} is eating  .. ")
                    time.sleep(0.4)
     
    
dp = DiningPhilosopher(5)
t = [myThread(dp.eat, i).start() for i in range(5)]

    
                

Philosopher 0 is eating  .. 
Philosopher 2 is eating  .. 
Philosopher 2 is eating  .. Philosopher 0 is eating  .. 

Philosopher 0 is eating  .. 
Philosopher 2 is eating  .. 
Philosopher 0 is eating  .. Philosopher 2 is eating  .. 

Philosopher 0 is eating  .. Philosopher 2 is eating  .. 

Philosopher 3 is eating  .. 
Philosopher 1 is eating  .. 
Philosopher 3 is eating  .. Philosopher 1 is eating  .. 

Philosopher 1 is eating  .. Philosopher 3 is eating  .. 

Philosopher 1 is eating  .. Philosopher 3 is eating  .. 

Philosopher 3 is eating  .. Philosopher 1 is eating  .. 

Philosopher 4 is eating  .. 
Philosopher 4 is eating  .. 
Philosopher 4 is eating  .. 
Philosopher 4 is eating  .. 
Philosopher 4 is eating  .. 


### Fizz Buzz Multithreaded Leetcode

In [11]:
class FizzBuzz:
    def __init__(self, n: int):
        self.n = n
        self.f,self.b,self.fb = [Semaphore(0) for _ in range(3)]
        self.num = Semaphore(1) # kind of mutex 
        self.i = 1
        
    def release(self):
        self.i += 1
        if self.i > self.n:
            self.fb.release()
            self.b.release()
            self.f.release()
            self.num.release()
            return
        if self.i%3==0 and self.i%5==0:
            self.fb.release()
        elif self.i%3==0:
            self.f.release()
        elif self.i%5==0:
            self.b.release()
        else:
            self.num.release()
        

    # printFizz() outputs "fizz"
    def fizz(self, action) -> None:
        while True:
            self.f.acquire()
            if self.i > self.n: break
            action()
            self.release()

    # printBuzz() outputs "buzz"
    def buzz(self, action) -> None:
    	while True:
            self.b.acquire()
            if self.i > self.n: break
            action()
            self.release()

    # printFizzBuzz() outputs "fizzbuzz"
    def fizzbuzz(self, action) -> None:
        while True:
            self.fb.acquire()
            if self.i > self.n: break
            action()
            self.release()

    # printNumber(x) outputs "x", where x is an integer.
    def number(self, action) -> None:
        while True:
            self.num.acquire()
            if self.i > self.n: break
            action(self.i)
            self.release()


s = FizzBuzz(5)
myThread(s.fizz, lambda : print("Fizz", end = " "))
myThread(s.buzz, lambda : print("Buzz", end = " "))
myThread(s.fizzbuzz, lambda : print("FizzBuzz", end=" "))
myThread(s.number, lambda a: print(a, end = " "))
None

1 2 Fizz 4 Buzz 

In [5]:
a = 10
ans = 1
while True:
    temp = ans
    ans = (2*ans   + a/ans/ans)/3
    if abs(temp - ans) < 0.00001:
        break
print(ans)

2.154434690031884


In [8]:
10**(1/3)

2.154434690031884